In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from nltk import RegexpTokenizer, pos_tag, sent_tokenize, word_tokenize
from pandas.io.json import json_normalize

import json
import pandas as pd
import re

In [8]:
data = []
with open('dataset/SQuAD/train-v2.0.json') as f:
    json_data = json.load(f)['data']

    for i in range(len(json_data)):
        json_data_i = json_data[i]['paragraphs']
        
        for j in range(1):
            paragraph = json_data_i[j]['context']
            
            data.append([sentence for sentence in sent_tokenize(paragraph)])

# Sentence Selection

### Sentence importance sorting

In [360]:
idx = 131

for i in range(len(data[idx])):
    print(i, data[idx][i])

0 Beginning in 1689, the colonies became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754–63) when Britain was victorious over all the French colonies in North America.
1 This final war was to give thousands of colonists, including Virginia colonel George Washington, military experience which they put to use during the American Revolutionary War.


In [361]:
from sentence_selection import MultiWordPhraseExtractor, preprocess

rank = MultiWordPhraseExtractor().summarize(preprocess(' '.join(data[idx])), top=3)
rank

[1, 0]

# Sentence Parsing

In [362]:
from question_generation import Sentence

In [363]:
sentence = Sentence(data[idx][0])
sentence.sentence

"Beginning in 1689, the colonies became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754–63) when Britain was victorious over all the French colonies in North America."

In [364]:
sentence.parse_constituents()
print(list(map(lambda x: print(x), sentence.constituent_tree)))

(S
  (PP (VBG Beginning) (PP (IN in) (NP (CD 1689))))
  (, ,)
  (NP (DT the) (NNS colonies))
  (VP
    (VBD became)
    (NP
      (NP
        (ADJP
          (VBN involved)
          (PP
            (IN in)
            (NP
              (NP (DT a) (NN series))
              (PP
                (IN of)
                (NP
                  (NP (NNS wars))
                  (PP
                    (IN between)
                    (NP
                      (NNP Great)
                      (NNP Britain)
                      (CC and)
                      (NNP France)))))
              (PP
                (IN for)
                (NP
                  (NP (NN control))
                  (PP
                    (IN of)
                    (NP
                      (NP (NNP North) (NNP America))
                      (, ,)
                      (SBAR
                        (WHNP
                          (WHNP (DT the) (RBS most) (JJ important))
                          (WHPP (IN of) (WHN

In [365]:
sentence.extract_named_entities()
print(sentence.ner_classes)
print(sentence.named_entities)
print(sentence.named_entities_dict)

[('Beginning', 'O'), ('in', 'O'), ('1689', 'O'), ('the', 'O'), ('colonies', 'O'), ('became', 'O'), ('involved', 'O'), ('in', 'O'), ('a', 'O'), ('series', 'O'), ('of', 'O'), ('wars', 'O'), ('between', 'O'), ('Great', 'O'), ('Britain', 'B-LOC'), ('and', 'O'), ('France', 'B-LOC'), ('for', 'O'), ('control', 'O'), ('of', 'O'), ('North', 'B-LOC'), ('America', 'I-LOC'), ('the', 'O'), ('most', 'O'), ('important', 'O'), ('of', 'O'), ('which', 'O'), ('were', 'O'), ('Queen', 'O'), ('Anne', 'B-PER'), ('s', 'O'), ('War', 'O'), ('in', 'O'), ('which', 'O'), ('the', 'O'), ('British', 'B-MISC'), ('conquered', 'O'), ('French', 'B-MISC'), ('colony', 'O'), ('Acadia', 'B-LOC'), ('and', 'O'), ('the', 'O'), ('final', 'O'), ('French', 'B-MISC'), ('and', 'O'), ('Indian', 'B-MISC'), ('War', 'I-MISC'), ('1754', 'O'), ('63', 'O'), ('when', 'O'), ('Britain', 'B-LOC'), ('was', 'O'), ('victorious', 'O'), ('over', 'O'), ('all', 'O'), ('the', 'O'), ('French', 'B-MISC'), ('colonies', 'O'), ('in', 'O'), ('North', 'B-LOC

In [366]:
# sentence.parse_srl()
# for clause in sentence.srl:
#     print(clause)

{'V': 'Beginning', 'AM-TMP': 'in 1689'}
{'A1': 'the colonies', 'V': 'became', 'A2': "involved in a series of wars between Great Britain and France for control of North America , the most important of which were Queen Anne 's War , in which the British conquered French colony Acadia , and the final French and Indian War ( 1754 63 ) when Britain was victorious over all the French colonies in North America"}
{'A1': 'the colonies', 'V': 'involved', 'A2': "in a series of wars between Great Britain and France for control of North America , the most important of which were Queen Anne 's War , in which the British conquered French colony Acadia , and the final French and Indian War ( 1754 63 )"}
{'V': 'conquered', 'A1': 'French colony Acadia'}


# Gap Formulation

In [367]:
from question_generation import GapSelector

In [368]:
gapper = GapSelector(sentence)

candidates = gapper.process()
for i, parent in enumerate(gapper.parents):
    print(i, parent)

0 (S
  (PP (VBG Beginning) (PP (IN in) (NP (CD 1689))))
  (, ,)
  (NP (DT the) (NNS colonies))
  (VP
    (VBD became)
    (NP
      (NP
        (ADJP
          (VBN involved)
          (PP
            (IN in)
            (NP
              (NP (DT a) (NN series))
              (PP
                (IN of)
                (NP
                  (NP (NNS wars))
                  (PP
                    (IN between)
                    (NP
                      (NNP Great)
                      (NNP Britain)
                      (CC and)
                      (NNP France)))))
              (PP
                (IN for)
                (NP
                  (NP (NN control))
                  (PP
                    (IN of)
                    (NP
                      (NP (NNP North) (NNP America))
                      (, ,)
                      (SBAR
                        (WHNP
                          (WHNP (DT the) (RBS most) (JJ important))
                          (WHPP (IN of) (W

In [369]:
print("Sentence: {}\n".format(sentence.sentence))
for candidate in candidates:
    print("Q: {}\nA: {}\n".format(candidate['question'], candidate['answer']))

Sentence: Beginning in 1689, the colonies became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754–63) when Britain was victorious over all the French colonies in North America.

Q: Beginning in _____, the colonies became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754–63) when Britain was victorious over all the French colonies in North America.
A: 1689

Q: Beginning in 1689, _____ became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian W

In [379]:
from nltk.stem import WordNetLemmatizer

from simple_nlg import init_simple_nlg

gateway = init_simple_nlg()
lexicon = gateway.jvm.Lexicon.getDefaultLexicon()
nlg_factory = gateway.jvm.NLGFactory(lexicon)
realiser = gateway.jvm.Realiser(lexicon)

wnl = WordNetLemmatizer()

blank = '_____'

for candidate in gapper.candidates:
    entity = candidate['entity']
    parent = candidate['parent']
    answer = candidate['answer']
    question = candidate['question']
    
    # Subject
    if entity.parent().label() == 'S':
        if answer in sentence.named_entities_dict['PER']:
            question = question.replace(blank, "who")
        else:
            question = question.replace(blank, "what")
        
        q = list(question)
        q[0] = q[0].upper()
        if q[-1] == '.':
            q[-1] = '?'
        else:
            q.append('?')
        question = ''.join(q)
        
    # Object
    elif entity.parent().parent().label() == 'S':
        p = nlg_factory.createClause()
        p.setObject(answer)
        
        for child in parent:
            # Subject
            if child.label() == 'NP':
                p.setSubject(' '.join(child.leaves()))
            
            # Verb
            elif child.label() == 'VP':
                for gchild in child:
                    if gchild.label() in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                        p.setVerb(wnl.lemmatize(gchild.leaves()[0], pos='v'))
                        if gchild.label() in ['VB', 'VBG', 'VBP', 'VBZ']:
                            p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PRESENT)
                        elif gchild.label() in ['VBD', 'VBN']:
                            p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PAST)
                        break
                        
            elif child.label() == 'PP':
                p.addComplement(' '.join(child.leaves()))
                        
        if answer in sentence.named_entities_dict['PER']:
            p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHO_OBJECT)
        else:
            p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHAT_OBJECT)
            
        question = realiser.realiseSentence(p)
        
    elif entity.parent().label() == 'PP':
        p = nlg_factory.createClause()
        
        for child in parent:
            # Subject
            if child.label() == 'NP':
                p.setSubject(' '.join(child.leaves()))
            
            # Verb
            elif child.label() == 'VP':
                for gchild in child:
                    if gchild.label() in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
                        p.setVerb(wnl.lemmatize(gchild.leaves()[0], pos='v'))
                        if gchild.label() in ['VB', 'VBG', 'VBP', 'VBZ']:
                            p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PRESENT)
                        elif gchild.label() in ['VBD', 'VBN']:
                            p.setFeature(gateway.jvm.Feature.TENSE, gateway.jvm.Tense.PAST)
                    elif gchild.label() == 'NP':
                        p.setObject(' '.join(gchild.leaves()))
                        
            elif child.label() == 'PP':
                p.addComplement(' '.join(child.leaves()))
                
        p.setFeature(gateway.jvm.Feature.INTERROGATIVE_TYPE, gateway.jvm.InterrogativeType.WHO_INDIRECT_OBJECT)
        question = realiser.realiseSentence(p)\
            .replace(answer, '')\
            .replace('  ', ' ')\
            .replace('to?', '?')\
            .replace(' ?', '?')
        question = re.sub(r"-LRB- (.*) -RRB-", r"(\1)", question)
        
        if check_in_dict(answer, sentence.named_entities_dict, 'PER'):
            pass
        elif check_in_dict(answer, sentence.named_entities_dict, 'LOC'):
            question = question.replace('Who', 'Where')
        else:
            question = question.replace('Who', 'What')
            
    print("Q: {}\nA: {}\n".format(question, candidate['answer']))
        

Q: What did the colonies become involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne 's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754 -- 63) when Britain was victorious over all the French colonies in North America Beginning in?
A: 1689

Q: Beginning in 1689, what became involved in a series of wars between Great Britain and France for control of North America, the most important of which were Queen Anne's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754–63) when Britain was victorious over all the French colonies in North America?
A: the colonies

Q: Where did the colonies become involved in a series of wars between for control of North America, the most important of which were Queen Anne 's War, in which the British conquered French colony Acadia, and the final French and Indian War (1754 -- 63) whe

In [371]:
def check_in_dict(answer, ne_dict, key):
    if answer in ne_dict[key]:
        return True
    
    for word in answer.split(' '):
        if word in ne_dict[key]:
            return True
    return False    

# Gap Creation

# Question Formation